<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/BB/bb_exploring_t5v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import json

# Make longer output readable without scrolling
from pprint import pprint

# Stop warning messages from showing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 5.2 MB/s 


In [ ]:
!pip install -q transformers

     |████████████████████████████████| 5.3 MB 5.5 MB/s 
     |████████████████████████████████| 163 kB 32.1 MB/s 
     |████████████████████████████████| 7.6 MB 39.3 MB/s 


In [ ]:
!pip install -q datasets

     |████████████████████████████████| 441 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 55.3 MB/s 
     |████████████████████████████████| 115 kB 46.7 MB/s 
     |████████████████████████████████| 127 kB 47.2 MB/s 


In [ ]:
!pip install -q evaluate
import evaluate

     |████████████████████████████████| 72 kB 953 kB/s 


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import list_datasets, load_dataset_builder, get_dataset_config_names, load_dataset, load_from_disk


In [ ]:
def summarize_dataset (dataset, config=None):
    builder = load_dataset_builder(dataset, config)
    print(f"Description:\n {builder.info.description}")
    print(f"Features:")
    pprint(builder.info.features)
    return

### Load SQuAD dataset from Gdrive

In [ ]:
data_squad = load_from_disk("/content/drive/MyDrive/w266 NLP Final Project/Data/squad.hf")

In [ ]:
data_squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
# Look at first example
pprint(data_squad['train'][0])

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did t

### T5v1.1 Model

Sources: https://huggingface.co/docs/transformers/model_doc/t5
https://huggingface.co/docs/transformers/model_doc/t5v1.1

In [ ]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

model = TFT5ForConditionalGeneration.from_pretrained("google/t5-v1_1-base")
tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")


# model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
# tokenizer = T5Tokenizer.from_pretrained("t5-base")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/t5-v1_1-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
model.summary()

Model: "tft5_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (TFSharedEmbeddings)  multiple                 24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  84954240  
                                                                 
 decoder (TFT5MainLayer)     multiple                  113275008 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247,577,856
Trainable params: 247,577,856
Non-trainable params: 0
_________________________________________________________________


### Input Structure

The input data should be a single list of dictionaries (or path to a JSON file containing the same). A dictionary represents a single context and its associated questions.

Each such dictionary contains two attributes, the "context" and "qas".

context: The paragraph or text from which the question is asked.
qas: A list of questions and answers (format below).
Questions and answers are represented as dictionaries. Each dictionary in qas has the following format.

id: (string) A unique ID for the question. Should be unique across the entire dataset.

question: (string) A question.

answers: (list) The list of correct answers to the question.

Source: https://simpletransformers.ai/docs/qa-data-formats/

In [ ]:
# Shuffle the dataset and take a handful of examples

count=25
sample=data_squad['train'].shuffle(seed=1962).select(range(count))
df=pd.DataFrame()
df['answer'] = [answer['text'][0] for answer in sample['answers']]
df['context'] = sample['context']
df['question'] = sample['question']

df

,answer,context,question
0,biotech companies,"Prior to moving its headquarters to Chicago, a...",What type of businesses did Nickles want to at...
1,Tytus Woyciechowski,Four boarders at his parents' apartments becam...,To whom did Chopin reveal in letters which par...
2,the Endangered Species Committee,The question to be answered is whether a liste...,"If a species may be harmed, who holds final sa..."
3,China,"In Asian countries such as China, Korea, and J...",What country has the dog as part of its 12 ani...
4,45 years,Saint Athanasius of Alexandria (/ˌæθəˈneɪʃəs/;...,How long did his episcopate last?
5,"Cold War, First Gulf War, Kosovo War","Since 1947, Canadian military units have parti...",What are some of the wars the Canadian Militar...
6,Buddha,Tibet has various festivals that are commonly ...,What is worshipped during Tibet's various fest...
7,9.3%,"From 2001 to 2008, Mac sales increased continu...",What was Apples market share of all computer s...
8,Improvisation,Improvisation stands at the centre of Chopin's...,What is central to Chopin's process?
9,1861,"Alfred North Whitehead was born in Ramsgate, K...",What year was Whitehead born?


Source: https://huggingface.co/ZhangCheng/T5v1.1-Base-Fine-Tuned-for-Question-Generation?text=%3Canswer%3E+thousands+%3Ccontext%3E+Transformers+provides+thousands+of+pre-trained+models+to+perform+tasks+on+different+modalities+such+as+text%2C+vision%2C+and+audio.


Example 1

In [ ]:
# Sample text to test from dataset
answer = df.answer[0]
context = df.context[0]
task = "qg"

input_text = f"{task} answer: {answer} </s> context: {context}"

output_text = df.question[0]

In [ ]:
# Input text is answer and context
input_text

"qg answer: biotech companies </s> context: Prior to moving its headquarters to Chicago, aerospace manufacturer Boeing (#30) was the largest company based in Seattle. Its largest division is still headquartered in nearby Renton, and the company has large aircraft manufacturing plants in Everett and Renton, so it remains the largest private employer in the Seattle metropolitan area. Former Seattle Mayor Greg Nickels announced a desire to spark a new economic boom driven by the biotechnology industry in 2006. Major redevelopment of the South Lake Union neighborhood is underway, in an effort to attract new and established biotech companies to the city, joining biotech companies Corixa (acquired by GlaxoSmithKline), Immunex (now part of Amgen), Trubion, and ZymoGenetics. Vulcan Inc., the holding company of billionaire Paul Allen, is behind most of the development projects in the region. While some see the new development as an economic boon, others have criticized Nickels and the Seattle C

In [ ]:
# Desired output text is a question
output_text

'What type of businesses did Nickles want to attract to Seattle?'

In [ ]:
# Tokenize input text
inputs = tokenizer(input_text, max_length=1024, truncation=True, return_tensors="tf")

In [ ]:
# Check tokenized inputs
inputs

{'input_ids': <tf.Tensor: shape=(1, 318), dtype=int32, numpy=
array([[    3,  1824,   122,  1525,    10,  2392,  3470,   688,     1,
         2625,    10,  6783,    12,  1735,   165, 13767,    12,  3715,
            6, 28674,  4818, 21430,    41,  4663,  1458,    61,    47,
            8,  2015,   349,     3,   390,    16,  8854,     5,    94,
            7,  2015,  4889,    19,   341,     3, 27630,    16,  4676,
         9405,   106,     6,    11,     8,   349,    65,   508,  6442,
         3732,  2677,    16,  6381,    15,    17,    17,    11,  9405,
          106,     6,    78,    34,  3048,     8,  2015,  1045,  6152,
           16,     8,  8854, 25233,   616,     5, 18263,  8854, 12394,
        11859, 29005,     7,  2162,     3,     9,  3667,    12, 13233,
            3,     9,   126,  1456, 13997,  6737,    57,     8,  2392,
        18485,   681,    16, 15066,  9236,     3,    60, 19677,    13,
            8,  1013,  2154,  3545,  5353,    19, 18953,     6,    16,
           46, 

In [ ]:
# Generate question output
outputs = model.generate(input_ids = inputs['input_ids'],
                         attention_mask = inputs['attention_mask'])

question_output = pprint(tokenizer.batch_decode(outputs, 
                                                skip_special_tokens=True, 
                                                clean_up_tokenization_spaces=True
                                                )[0], compact=True)

question_output

'. Boeing (#30) was founded in Seattle in 1931. Boeing (#'


Example 6

In [ ]:
# Sample text to test from dataset
answer = df.answer[6]
context = df.context[6]
task = "qg"

input_text = f"{task} answer: {answer} </s> context: {context}"

output_text = df.question[6]

In [ ]:
# Input text is answer and context
input_text

"qg answer: Buddha </s> context: Tibet has various festivals that are commonly performed to worship the Buddha[citation needed] throughout the year. Losar is the Tibetan New Year Festival. Preparations for the festive event are manifested by special offerings to family shrine deities, painted doors with religious symbols, and other painstaking jobs done to prepare for the event. Tibetans eat Guthuk (barley noodle soup with filling) on New Year's Eve with their families. The Monlam Prayer Festival follows it in the first month of the Tibetan calendar, falling between the fourth and the eleventh days of the first Tibetan month. It involves dancing and participating in sports events, as well as sharing picnics. The event was established in 1049 by Tsong Khapa, the founder of the Dalai Lama and the Panchen Lama's order."

In [ ]:
# Desired output text is a question
output_text

"What is worshipped during Tibet's various festibals?"

In [ ]:
# Tokenize input text
inputs = tokenizer(input_text, max_length=1024, truncation=True, return_tensors="tf")

In [ ]:
# Check tokenized inputs
inputs

{'input_ids': <tf.Tensor: shape=(1, 188), dtype=int32, numpy=
array([[    3,  1824,   122,  1525,    10, 21554,     1,  2625,    10,
        28920,    65,   796, 14856,    24,    33,  5871,  3032,    12,
         7373,     8, 21554,  6306, 13903,   906,   908,  1019,     8,
          215,     5,  3144,   291,    19,     8, 26488,   368,  2929,
         3397,     5,  1266,  1893,  1628,    21,     8, 15723,   605,
           33,  6571,    15,    26,    57,   534, 11571,    12,   384,
        30258,    20,  2197,     6,  7445,  3377,    28,  4761, 13619,
            6,    11,   119,  1406,     7, 14867,  2476,   612,    12,
         2967,    21,     8,   605,     5, 26488,     7,     3,  1544,
         2846,   189,  1598,    41,  1047,  1306,     3,    29, 14957,
         5759,    28,    14,    53,    61,    30,   368,  2929,    31,
            7, 11566,    28,    70,  1791,     5,    37,  2963,    40,
          265, 23535,  3397,  6963,    34,    16,     8,   166,   847,
           13, 

In [ ]:
# Generate question output
outputs = model.generate(input_ids = inputs['input_ids'],
                         attention_mask = inputs['attention_mask'])

question_output = pprint(tokenizer.batch_decode(outputs, 
                                                skip_special_tokens=True, 
                                                clean_up_tokenization_spaces=True
                                                )[0], compact=True)

question_output

('the Tibetan New Year. The Tibetan New Year is celebrated in the month of '
 'January.')


### Try training the model

Source: https://huggingface.co/docs/transformers/model_doc/t5#transformers.TFT5ForConditionalGeneration

In [ ]:
# Come up with a prompt for T5 model
prompt = 'question_gen: '
# Add "answer: " to each answer in list
# answer = ["question_gen: answer: " + a for a in df.answer]
# Add "context: " to each context in list
context = ["context: " + c for c in df.context]
# Add "question: " to each question in list
question = ["question: " + q for q in df.question]

# Input text will be prompt + answer + context
input_txt = [prompt + x for x in context]

# Output text will be questions
output_txt = question

input_txt

["question_gen: context: Prior to moving its headquarters to Chicago, aerospace manufacturer Boeing (#30) was the largest company based in Seattle. Its largest division is still headquartered in nearby Renton, and the company has large aircraft manufacturing plants in Everett and Renton, so it remains the largest private employer in the Seattle metropolitan area. Former Seattle Mayor Greg Nickels announced a desire to spark a new economic boom driven by the biotechnology industry in 2006. Major redevelopment of the South Lake Union neighborhood is underway, in an effort to attract new and established biotech companies to the city, joining biotech companies Corixa (acquired by GlaxoSmithKline), Immunex (now part of Amgen), Trubion, and ZymoGenetics. Vulcan Inc., the holding company of billionaire Paul Allen, is behind most of the development projects in the region. While some see the new development as an economic boon, others have criticized Nickels and the Seattle City Council for pan

In [ ]:
output_txt

['question: What type of businesses did Nickles want to attract to Seattle?',
 'question: To whom did Chopin reveal in letters which parts of his work were about the singing student he was infatuated with?',
 'question: If a species may be harmed, who holds final say on whether the project may proceed?',
 'question: What country has the dog as part of its 12 animals that represent years?',
 'question: How long did his episcopate last?',
 'question: What are some of the wars the Canadian Military was involved in?',
 "question: What is worshipped during Tibet's various festibals?",
 'question: What was Apples market share of all computer shipments in 2011?',
 "question: What is central to Chopin's process?",
 'question: What year was Whitehead born?',
 'question: What percentage of plant and animal species extinction is predicted in the 21st century?',
 'question: If Bob realized he needed to mail his mother a letter after seeing the postal office, which kind of memory did he trigger?',


In [ ]:
# Training
inputs = tokenizer(input_txt, max_length=1024, padding=True, truncation=True, return_tensors="tf").input_ids
labels = tokenizer(output_txt, max_length=1024, padding=True, truncation=True, return_tensors="tf").input_ids
outputs = model(inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
# Inference
inputs = tokenizer(
    "In Asian countries such as China, Korea, and Japan, dogs are viewed as kind protectors. The role of the dog in Chinese mythology includes a position as one of the twelve animals which cyclically represent years (the zodiacal dog).", return_tensors="tf"
).input_ids  # Batch size 1

outputs = model.generate(inputs)

print(tokenizer.decode(outputs[0]
                       , skip_special_tokens=True))

/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py:1695: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


the zodiacal dog. Dogs are protectors of the home. Dogs are


In [ ]:
# Not generate model with more sophistication (kind of)
soph_outputs = model.generate(inputs,
                              num_beams=2,
                              no_repeat_ngram_size=1,
                              min_length=20,
                              max_length=40)
                             
print([tokenizer.decode(x, skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False) for x in soph_outputs])

['the zodiacal dog. Dogs are viewed as protector of humans and animals in general,?']


### Try running more data through model

In [ ]:
prompt = 'qg: '
answer = df.answer
context = df.context

input_txt = [prompt + sequence for sequence in answer + context]

output_txt = df.question

In [ ]:
input_txt

["qg: biotech companiesPrior to moving its headquarters to Chicago, aerospace manufacturer Boeing (#30) was the largest company based in Seattle. Its largest division is still headquartered in nearby Renton, and the company has large aircraft manufacturing plants in Everett and Renton, so it remains the largest private employer in the Seattle metropolitan area. Former Seattle Mayor Greg Nickels announced a desire to spark a new economic boom driven by the biotechnology industry in 2006. Major redevelopment of the South Lake Union neighborhood is underway, in an effort to attract new and established biotech companies to the city, joining biotech companies Corixa (acquired by GlaxoSmithKline), Immunex (now part of Amgen), Trubion, and ZymoGenetics. Vulcan Inc., the holding company of billionaire Paul Allen, is behind most of the development projects in the region. While some see the new development as an economic boon, others have criticized Nickels and the Seattle City Council for pande

In [ ]:
output_txt

0     What type of businesses did Nickles want to at...
1     To whom did Chopin reveal in letters which par...
2     If a species may be harmed, who holds final sa...
3     What country has the dog as part of its 12 ani...
4                     How long did his episcopate last?
5     What are some of the wars the Canadian Militar...
6     What is worshipped during Tibet's various fest...
7     What was Apples market share of all computer s...
8                  What is central to Chopin's process?
9                         What year was Whitehead born?
10    What percentage of plant and animal species ex...
11    If Bob realized he needed to mail his mother a...
12        What kind of giant insect form had wingspans?
13    What is an example of a collection of classic ...
14               What land area is west of Puget Sound?
15    How were trade goods transported in southern M...
16    From where is the temperature of a glacier mea...
17                             When was Shen Kuo

In [ ]:
# Tokenize input text
# Add padding to have batched tensors with same length
inputs = tokenizer(input_txt, 
                   padding = "longest",
                   max_length=1024, 
                   truncation=True, 
                   return_tensors="tf")

In [ ]:
# Generate question output
outputs = model.generate(input_ids = inputs['input_ids'],
                         attention_mask = inputs['attention_mask'])

question_output = pprint(tokenizer.batch_decode(outputs, 
                                                skip_special_tokens=True, 
                                                clean_up_tokenization_spaces=True
                                                )[0], compact=True)

question_output

/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py:1695: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


'.com..com..com..com..com'


The T5 model is instructed to perform a particular task by adding a prefix to the start of an input sequence. The prefix for a specific task may be any arbitrary text as long as the same prefix is prepended whenever the model is supposed to execute the given task.

Source: https://simpletransformers.ai/docs/t5-specifics/#:~:text=The%20T5%20model%20is%20instructed,to%20execute%20the%20given%20task.&text=binary%20classification

### Try two inputs for

Source: https://huggingface.co/docs/transformers/model_doc/t5#training

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import torch

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
# the following 2 hyperparameters are task-specific
max_source_length = 1024
max_target_length = 500

# Suppose we have the following 2 training examples:
input_sequence_1 = df.context[0]
output_sequence_1 = df.question[0]

input_sequence_2 = df.context[6]
output_sequence_2 = df.question[6]

# encode the inputs
task_prefix = "qg: "
input_sequences = [input_sequence_1, input_sequence_2]

encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

# encode the targets
target_encoding = tokenizer(
    [output_sequence_1, output_sequence_2],
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="tf",
)
labels = target_encoding.input_ids

# replace padding token id's of the labels by -100 so it's ignored by the loss
# labels[labels == tokenizer.pad_token_id] = -100

# forward pass
# loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
# loss.item()


# outputs = model.generate(input_ids = inputs['input_ids'],
#                          attention_mask = inputs['attention_mask'])

# outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

# question_output = pprint(tokenizer.batch_decode(outputs, 
#                                                 skip_special_tokens=True, 
#                                                 clean_up_tokenization_spaces=True,
#                                                # max_new_tokens=60
#                                                 )[0], compact=True)

# question_output


output_sequences = model.generate(
    input_ids=encoding["input_ids"],
    attention_mask=encoding["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
    no_repeat_ngram_size=2,
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))


['. qg: Boeing is the second largest employer in the Seattle area. Boeing was', '. qg:  :::::.']


In [ ]:
# Generate Summary
output_sequences = model.generate(input_ids=encoding["input_ids"],
                                  attention_mask=encoding["attention_mask"], 
                              num_beams=1,
                              no_repeat_ngram_size=2,
                              min_length=10,
                              max_length=20)

pprint(tokenizer.batch_decode(output_sequences, 
                              skip_special_tokens=True, 
                              clean_up_tokenization_spaces=False)[0], 
       #compact=True
       )

'. qg: Boeing is the second largest employer in the Seattle area. Boeing was'


prefix: A string indicating the task to perform.

input_text: The input text sequence.

target_text: The target sequence.